In [1]:
import numpy as np
import pandas as pd
import os

# 10人分の平均値を使用したもの
1要因分析

In [47]:

col = ['number', 'graph']
study_df = pd.DataFrame(index=[], columns=col)

for data in os.listdir('data'):

    df = pd.read_csv('data/' + data)

    df_number = df[df['sender'] == 'Stimulus Number']
    number_mean = df_number['duration'].mean()

    df_graph = df[df['sender'] == 'Stimulus Graph']
    graph_mean = df_graph['duration'].mean()

    case = np.array([number_mean, graph_mean])

    case_ser = pd.Series(case, index=study_df.columns)
    study_df = study_df.append(case_ser,ignore_index=True)

# データを行列形式に変換する
data = study_df.values

# 全平均
all_mean = np.full(data.shape, data.mean())

# 列の効果 列ごとの平均ー全平均
col_effect = data.mean(axis=0) - all_mean

# 誤差 データー列の平均
err = data - data.mean(axis=0)

# 列の不偏分散
phi_1 = data.shape[1] - data.mean().size
V_1 = np.sum(np.square(col_effect)) / phi_1

# 誤差の不偏分散
phi_2 = data.size - data.mean(axis=0).size
V_2 = np.sum(np.square(err)) / phi_2

# F値 V1/V2
F = V_1 / V_2

# 検定
from scipy.stats import f

five = f.ppf(0.95, phi_1,phi_2)
one = f.ppf(0.99, phi_1,phi_2)

print(phi_1, V_1)
print(phi_2, V_2)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('F = ', F)

1 1127127.4356155372
18 54297.58603362514
上側確率 5% 4.413873419170566
上側確率 1% 8.285419555099654
F =  20.75833417193787


# 10人分の Raw Data を使用したもの
１要因分析

In [27]:
df = pd.DataFrame();
for csv in os.listdir('data'):
    data = pd.read_csv('data/' + csv)
    data['Person'] = csv[0]
    df = df.append(data)

df_number = df[df['sender'] == 'Stimulus Number']
df_graph = df[df['sender'] == 'Stimulus Graph']

number = df_number['duration'].values
graph = df_graph['duration'].values
study_df = pd.DataFrame({'number':number, 'graph':graph})

study_df.to_csv('study_df_raw_anova1.csv')
# データを行列形式に変換する
data = study_df.values

# 全平均
all_mean = np.full(data.shape, data.mean())

# 列の効果 列ごとの平均ー全平均
col_effect = data.mean(axis=0) - all_mean

# 誤差 データー列の平均
err = data - all_mean - col_effect

# 列の不偏分散
phi_1 = data.shape[1] - data.mean().size
V_1 = np.sum(np.square(col_effect)) / phi_1

# 誤差の不偏分散
phi_2 = data.size - data.mean(axis=0).size
V_2 = np.sum(np.square(err)) / phi_2

# F値 V1/V2
F = V_1 / V_2

# 検定
from scipy.stats import f

five = f.ppf(0.95, phi_1,phi_2)
one = f.ppf(0.99, phi_1,phi_2)

print(phi_1, V_1)
print(phi_2, V_2)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('F = ', F)


1 11271274.35615537
198 320282.3932302712
上側確率 5% 3.8888529328918806
上側確率 1% 6.764614632812903
F =  35.19167645300983


# 10人分の Raw Data を使用したもの

2要因分析
横：表示方法（数値、グラフ）2水準
縦：実験者（10名）10水準

In [54]:
df = pd.DataFrame();
for csv in os.listdir('data'):
    data = pd.read_csv('data/' + csv)
    data['Person'] = csv[0]
    df = df.append(data)

df_number = df[df['sender'] == 'Stimulus Number']
df_graph = df[df['sender'] == 'Stimulus Graph']

df1 = df_number.groupby('Person')['duration'].mean()
df1.name = 'number'

df2 = df_graph.groupby('Person')['duration'].mean()
df2.name = 'graph'

df3 = pd.concat([df1,df2], axis=1)

print(df3)
# df3.to_csv('df3.csv')

# データを行列形式に変換する
data = df3.values

# 全平均
all_mean = np.full(data.shape, data.mean())
# 列の効果 列ごとの平均 - 全平均
col_effect = data.mean(axis=0) - all_mean
# 行の効果 行ごとの平均 - 全平均
row_effect = data.mean(axis=1)[:,np.newaxis] - all_mean
# 誤差 データー列の平均
err = data - all_mean - col_effect - row_effect

# 列の不偏分散
phi_11 = data.shape[1] - data.mean().size
V_11 = np.sum(np.square(col_effect)) / phi_11
print(V_11)

# 行の不偏分散
phi_12 = data.shape[0] - data.mean().size
V_12 = np.sum(np.square(row_effect)) / phi_12
print(V_12)

# 誤差の不偏分散
phi_2 = phi_11 * phi_12
V_2 = np.sum(np.square(err)) / phi_2
print(V_2)

# 列のF値
F1 = V_11 / V_2

# 行のF値
F2 = V_12 / V_2

# 検定
from scipy.stats import f

five = f.ppf(0.95, phi_11,phi_12)
one = f.ppf(0.99, phi_11,phi_12)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('列のF = ', F1)
print('行のF = ', F2)

           number      graph
Person                      
A       1441.9959   961.4230
B       1323.3850  1007.3128
C       1732.4297   948.1771
D       1914.7384  1329.6288
E       1964.1775  1175.9615
F       1528.2822  1271.6602
G       1476.1134  1211.9644
H       1870.7885  1502.2920
I       1444.3888   715.4131
J       1551.7986  1376.3657
1127127.4356155372
81434.70467621187
27160.467391038444
上側確率 5% 5.117355029199227
上側確率 1% 10.561431047395386
列のF =  41.49882324880135
行のF =  2.99828068139509


# 数字の距離に注目して分析
数字セットの中に差が小さい組み合わせがあれば

In [28]:
def diff_func(x):
    n = np.sort(x[['num1','num2','num3','num4','num5']].values)
    m = np.min([n[4]-n[3], n[3]-n[2], n[2]-n[1], n[1]-n[0]])
    return m

df = pd.DataFrame();
for csv in os.listdir('data'):
    data = pd.read_csv('data/' + csv)
    data['Person'] = csv[0]
    df = df.append(data)

df_number = df[df['sender'] == 'Stimulus Number']
df_graph = df[df['sender'] == 'Stimulus Graph']

df_number['diff'] = df_number.apply(diff_func, axis=1)
df_graph['diff'] = df_graph.apply(diff_func, axis=1)

group = 'diff'
df1 = df_number.groupby(group)['duration'].mean()
df1.name = 'number'
df2 = df_graph.groupby(group)['duration'].mean()
df2.name = 'graph'

df3 = pd.concat([df1,df2], axis=1)

df3.to_csv('df3_diff.csv')
print(df3)

# データを行列形式に変換する
data = df3.values

# 全平均
all_mean = np.full(data.shape, data.mean())
# 列の効果 列ごとの平均 - 全平均
col_effect = data.mean(axis=0) - all_mean
# 行の効果 行ごとの平均 - 全平均
row_effect = data.mean(axis=1)[:,np.newaxis] - all_mean
# 誤差 データー列の平均
err = data - all_mean - col_effect - row_effect

# 列の不偏分散
phi_11 = data.shape[1] - data.mean().size
V_11 = np.sum(np.square(col_effect)) / phi_11
print(V_11)

# 行の不偏分散
phi_12 = data.shape[0] - data.mean().size
V_12 = np.sum(np.square(row_effect)) / phi_12
print(V_12)

# 誤差の不偏分散
phi_2 = phi_11 * phi_12
V_2 = np.sum(np.square(err)) / phi_2
print(V_2)

# 列のF値
F1 = V_11 / V_2

# 行のF値
F2 = V_12 / V_2

# 検定
from scipy.stats import f

five = f.ppf(0.95, phi_11,phi_12)
one = f.ppf(0.99, phi_11,phi_12)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('列のF = ', F1)
print('行のF = ', F2)

           number      graph
diff                        
1.0   1608.531650  1267.5357
2.0   1470.962933  1045.2094
3.0   2063.454400  1031.9520
4.0   1575.579200  1048.8391
5.0   1762.049000  1213.6365
825645.3324710225
29220.455892635597
36057.78104964416
上側確率 5% 7.708647422176786
上側確率 1% 21.197689584391295
列のF =  22.897840866421546
行のF =  0.810378648991324


# 数字の標準偏差に注目して分析

In [29]:
df = pd.DataFrame();
for csv in os.listdir('data'):
    data = pd.read_csv('data/' + csv)
    data['Person'] = csv[0]
    df = df.append(data)

df_number = df[df['sender'] == 'Stimulus Number']
df_graph = df[df['sender'] == 'Stimulus Graph']

group = 'std'
df1 = df_number.groupby(group)['duration'].mean()
df1.name = 'number'
df2 = df_graph.groupby(group)['duration'].mean()
df2.name = 'graph'

df3 = pd.concat([df1,df2], axis=1)
df3.to_csv('df3_std.csv')
print(df3)

# データを行列形式に変換する
data = df3.values

# 全平均
all_mean = np.full(data.shape, data.mean())
# 列の効果 列ごとの平均 - 全平均
col_effect = data.mean(axis=0) - all_mean
# 行の効果 行ごとの平均 - 全平均
row_effect = data.mean(axis=1)[:,np.newaxis] - all_mean
# 誤差 データー列の平均
err = data - all_mean - col_effect - row_effect

# 列の不偏分散
phi_11 = data.shape[1] - data.mean().size
V_11 = np.sum(np.square(col_effect)) / phi_11
print(V_11)

# 行の不偏分散
phi_12 = data.shape[0] - data.mean().size
V_12 = np.sum(np.square(row_effect)) / phi_12
print(V_12)

# 誤差の不偏分散
phi_2 = phi_11 * phi_12
V_2 = np.sum(np.square(err)) / phi_2
print(V_2)

# 列のF値
F1 = V_11 / V_2

# 行のF値
F2 = V_12 / V_2

# 検定
from scipy.stats import f

five = f.ppf(0.95, phi_11,phi_12)
one = f.ppf(0.99, phi_11,phi_12)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('列のF = ', F1)
print('行のF = ', F2)

              number      graph
std                            
4.270831   1615.7423  1017.4244
7.002857   2063.4544  1031.9520
7.071068   1818.5970  2101.3559
8.158431   1270.2243  1002.6831
8.366600   1175.1173   866.7857
8.447485   1751.0468  1221.4938
10.556515  1575.5792  1048.8391
11.689311  1486.7247  1047.3487
12.464349  1762.0490  1213.6365
18.117395  1729.5630   948.6794
1127127.435615537
132922.30312714222
59766.34184607866
上側確率 5% 5.117355029199227
上側確率 1% 10.561431047395386
列のF =  18.858899521043533
行のF =  2.224032775328099
